## การสร้างเรขาคณิตแบบจุด (Point Geometry)

หอไอเฟลเป็นหอคอยโครงเหล็กที่สร้างขึ้นในศตวรรษที่ 19 และถือเป็นหนึ่งในสัญลักษณ์ที่โดดเด่นที่สุดของกรุงปารีส  

ตำแหน่งของหอไอเฟลมีพิกัดดังนี้:
- ค่า x = 255422.6  
- ค่า y = 6250868.9

In [ ]:
# Import the Point geometry
from shapely.geometry import Point

# Construct a point object for the Eiffel Tower
eiffel_tower = Point(255422.6,6250868.9)

# Print the result
print(eiffel_tower)

## เมธอดเชิงพื้นที่ของ Shapely

เมื่อเรามีวัตถุ `Point` ของหอไอเฟลที่สร้างด้วย Shapely แล้ว  
เราสามารถใช้เมธอดต่าง ๆ ที่มีอยู่ในวัตถุเรขาคณิตนี้ เพื่อทำการคำนวณและวิเคราะห์เชิงพื้นที่ได้ เช่น

- การคำนวณระยะทาง (distance)
- การตรวจสอบความสัมพันธ์เชิงพื้นที่ (spatial relationship) ระหว่างวัตถุ

In [ ]:
import geopandas as gpd
from shapely.geometry import Point

districts = gpd.read_file("../../data/raw/paris_districts_utm.geojson")
district_montparnasse = districts.loc[52, "geometry"]

restaurants = gpd.read_file("../../data/processed/paris_restaurants.geojson")
resto = restaurants.loc[956, "geometry"]


print(eiffel_tower.within(district_montparnasse))
print(district_montparnasse.contains(resto))
print(eiffel_tower.distance(resto))

## หอไอเฟลตั้งอยู่ในเขตใดของปารีส?

กลับมาที่ตัวอย่างหอไอเฟลอีกครั้ง  
ในแบบฝึกหัดก่อนหน้า เราได้สร้างเรขาคณิตแบบจุด (Point) สำหรับตำแหน่งของหอไอเฟล และตรวจสอบแล้วว่ามันไม่ได้อยู่ในเขต Montparnasse

ตอนนี้ เราจะมาหาว่าหอไอเฟลตั้งอยู่ในเขต (district) ใดของกรุงปารีส

In [ ]:
# อ่านข้อมูลเขต
districts = gpd.read_file("../../data/raw/paris_districts_utm.geojson")

# พิกัดนี้เป็น Web Mercator (EPSG:3857)
eiffel = gpd.GeoSeries([Point(255422.6, 6250868.9)], crs="EPSG:3857")

# แปลงให้ตรงกับ CRS ของ districts (เช่น EPSG:32631)
eiffel = eiffel.to_crs(districts.crs)

mask = districts.contains(eiffel.geometry.squeeze())
print(districts.loc[mask, ["id", "district_name", "population"]])

## ร้านอาหารที่ใกล้ที่สุดอยู่ห่างเท่าไร?

ตอนนี้ เราอาจสนใจว่ามีร้านอาหารใดอยู่ใกล้หอไอเฟลบ้าง  
เพื่อสำรวจเรื่องนี้ เราจะทำการแสดงผลตำแหน่งของหอไอเฟล พร้อมทั้งร้านอาหารที่อยู่ในระยะไม่เกิน 1 กิโลเมตรจากจุดนั้น

In [ ]:
import matplotlib.pyplot as plt
import contextily as ctx
# อ่านข้อมูล
restaurants = gpd.read_file("../../data/processed/paris_restaurants.geojson")

# แก้ป้าย CRS ให้ถูก (ข้อมูลนี้เป็น EPSG:3857)
restaurants = restaurants.set_crs(epsg=3857, allow_override=True)

# สร้าง Eiffel Tower ใน 3857
eiffel_tower = gpd.GeoSeries(
    [Point(255422.6, 6250868.9)],
    crs="EPSG:3857"
)

# คำนวณระยะ
dist_eiffel = restaurants.distance(eiffel_tower.iloc[0])

print("Closest restaurant distance:", dist_eiffel.min())

# เลือกร้านในระยะ 1 km
restaurants_eiffel = restaurants[dist_eiffel < 1000]

# Plot
fig, ax = plt.subplots(figsize=(8,8))

restaurants_eiffel.plot(ax=ax, color="blue", markersize=10)
eiffel_tower.plot(ax=ax, color="red", markersize=100)

ctx.add_basemap(ax, crs=restaurants.crs)

ax.set_axis_off()
plt.show()

## ปารีส: การทำ Spatial Join ระหว่างเขตและสถานีจักรยาน

กลับมาที่ข้อมูลกรุงปารีสเกี่ยวกับเขตการปกครอง (districts) และสถานีจักรยานกันอีกครั้ง  

ในขั้นตอนนี้ เราจะใช้การทำ **Spatial Join** เพื่อระบุว่าสถานีจักรยานแต่ละแห่งตั้งอยู่ในเขตใดของปารีส

In [ ]:
stations = gpd.read_file("../../data/raw/paris_sharing_bike_stations_utm.geojson")

# Join the districts and stations datasets
joined = stations.sjoin(districts)

# Inspect the first five rows of the result
print(joined.head())

## แผนที่ความหนาแน่นของต้นไม้รายเขต (1)

โดยใช้ชุดข้อมูลต้นไม้ทั้งหมดในพื้นที่สาธารณะของกรุงปารีส เป้าหมายคือการสร้างแผนที่แสดงความหนาแน่นของต้นไม้ในแต่ละเขต (district)

เพื่อทำเช่นนั้น ขั้นแรกเราจำเป็นต้องทราบว่าในแต่ละเขตมีต้นไม้จำนวนกี่ต้น ซึ่งเราจะดำเนินการในแบบฝึกหัดนี้ จากนั้นในแบบฝึกหัดถัดไป จะนำผลลัพธ์ที่ได้ไปคำนวณความหนาแน่นและสร้างแผนที่ต่อไป

In [ ]:
# Read the trees and districts data
trees = gpd.read_file("../../data/raw/paris_trees_small.gpkg")
districts = gpd.read_file("../../data/raw/paris_districts_utm.geojson")

# Spatial join of the trees and districts datasets
joined = gpd.sjoin(trees, districts, predicate="within")

# Calculate the number of trees in each district
trees_by_district = joined.groupby("district_name").size()

# Convert the series to a DataFrame and specify column name
trees_by_district = trees_by_district.to_frame(name='n_trees')

# Inspect the result
print(trees_by_district.head())

## แผนที่ความหนาแน่นของต้นไม้รายเขต (2)

เมื่อเราได้จำนวนต้นไม้ในแต่ละเขตแล้ว ขั้นตอนถัดไปคือการสร้างแผนที่ที่แสดงสีของแต่ละเขตตามความหนาแน่นของต้นไม้

ก่อนอื่น เราต้องนำข้อมูลจำนวนต้นไม้ในแต่ละเขตที่คำนวณไว้ในขั้นตอนก่อนหน้า (`trees_by_district`) มารวม (merge) เข้ากับชุดข้อมูลเขต (`districts`)

โดยจะใช้ฟังก์ชัน `pd.merge()` เพื่อเชื่อมข้อมูลจากสอง DataFrame เข้าด้วยกัน โดยอาศัยคอลัมน์ที่มีค่าเหมือนกันเป็นตัวเชื่อม (common column)

In [ ]:
import pandas as pd
# Merge the 'districts' and 'trees_by_district' dataframes
districts_trees = pd.merge(districts, trees_by_district, on='district_name')

# Add a column with the tree density
districts_trees['n_trees_per_area'] = districts_trees['n_trees'] / districts_trees.geometry.area

# Make of map of the districts colored by 'n_trees_per_area'
districts_trees.plot(column="n_trees_per_area")
plt.show()

## แผนที่ Choropleth แบบช่วงเท่ากัน (Equal Interval)

ในแบบฝึกหัดก่อนหน้า เราได้สร้างแผนที่แสดงความหนาแน่นของต้นไม้ไปแล้ว ตอนนี้เมื่อเราเข้าใจเรื่อง choropleth มากขึ้น เราจะมาสำรวจการแสดงผลแบบนี้ให้ลึกขึ้น

ขั้นแรก เราจะเปรียบเทียบผลลัพธ์ระหว่างการใช้ “จำนวนต้นไม้ทั้งหมด” กับการใช้ “จำนวนต้นไม้ที่ปรับตามพื้นที่ของเขต” (ความหนาแน่นของต้นไม้)

ขั้นต่อมา เราจะสร้างแผนที่แบบ **Equal Interval** แทนการใช้สเกลสีแบบต่อเนื่อง โดยวิธีการจัดกลุ่มนี้จะแบ่งช่วงค่าของข้อมูลออกเป็นช่วงที่มีความกว้างเท่ากัน และกำหนดสีให้แต่ละช่วง

In [ ]:
# Print the first rows of the tree density dataset
print(districts_trees.head())

# Make a choropleth of the number of trees 
districts_trees.plot(column="n_trees", legend=True)
plt.show()

# Make a choropleth of the number of trees per area
districts_trees.plot(column="n_trees_per_area", legend=True)
plt.show()

# Make a choropleth of the number of trees 
districts_trees.plot(column="n_trees_per_area", scheme="equal_interval", legend=True)
plt.show()

## แผนที่ Choropleth แบบ Quantiles

ในแบบฝึกหัดนี้ เราจะสร้างแผนที่ความหนาแน่นของต้นไม้แบบ **Quantile**

อัลกอริทึมแบบ Quantile จะจัดเรียงค่าข้อมูลและแบ่งออกเป็นกลุ่มที่มีจำนวนข้อมูลเท่ากัน เพื่อกำหนดสีให้แต่ละกลุ่ม

ครั้งนี้ เราจะสร้างทั้งหมด **7 กลุ่ม** และใช้ชุดสี **YlGn colormap** กระจายครอบคลุมช่วงค่าทั้งหมดของข้อมูล

มีตัวแปร `n_trees_per_area` สำหรับวัดความหนาแน่นของต้นไม้ในแต่ละเขต  
(หมายเหตุ: ค่าตัวแปรนี้ถูกคูณด้วย 10,000)

In [ ]:
# Generate the choropleth and store the axis
ax = districts_trees.plot(column='n_trees_per_area', scheme='quantiles',
                          k=7, cmap='YlGn', legend=True)

# Remove frames, ticks and tick labels from the axis
ax.set_axis_off()
plt.show()

## เปรียบเทียบอัลกอริทึมการจัดกลุ่มข้อมูล (Classification Algorithms)

ในแบบฝึกหัดสุดท้ายนี้ คุณจะสร้างแผนที่หลายรูป (multi-map figure)  
เพื่อเปรียบเทียบสองแนวทางในการแสดงตัวแปรบนแผนที่ที่เราได้เรียนรู้มา

In [ ]:
# Set up figure and subplots
fig, axes = plt.subplots(nrows=2)

# Plot equal interval map
districts_trees.plot(column="n_trees_per_area", scheme="equal_interval", k=5, legend=True, ax=axes[0])
axes[0].set_title('Equal Interval')
axes[0].set_axis_off()

# Plot quantiles map
districts_trees.plot(column="n_trees_per_area", scheme="quantiles", k=5, legend=True, ax=axes[1])
axes[1].set_title('Quantiles')
axes[1].set_axis_off()

# Display maps
plt.show()